# 네이버 매장정보 크롤링

#### 자료 
https://www.localdata.go.kr/devcenter/dataDown.do?menuNo=20001

1. 관광식당, 일반음식점, 휴게음식점 데이터셋 다운로드(07_24_01_P_.csv.zip, 07_24_04_P_.csv.zip , 07_24_05_P_.csv.zip)
2. 다운로드한 파일들을 data 폴더에 넣어주세요. (압축풀지말고)

문제 
1. 강남구 매장이지만, 네이버로 검색하는 경우 강남구가 아닌 다른 지역의 매장명이 검색됨
2. 데이터 반영이 빠른편이지만, 네이버에 매장이 없는 경우가 있음 (없는 매장 : 맥도날드, 교촌, 굽네) 
3. 검색하면 전혀 다른 업체가 나오는 경우가 있음 -> 네이버 업태구분명을 크롤링해와서 참고하여 반영해야 
4. 나라에서 제공하는 data를 사용하는 경우 인기도를 알기 어려움. 아닌가. -> 네이버 지도 api를 사용해서 점수를 부여할까? 
5. 자료 업데이트 시기 문제 - data를 다운로드 받아서 사용하는 경우, 매장 정보는 계속 바뀐는데 업데이트가 늦잖아... 매장 정보의 변동이 생기 사람이 계속 업로드를 해야 하잖아.. 

In [3]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import zipfile
import os

In [4]:
# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
# driver = webdriver.Chrome(ChromeDriverManager().install())
# # 주소 이동
# url = 'https://map.naver.com/'
# driver.get(url)
# time.sleep(1)


# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
# 주소 이동
url = 'https://map.naver.com/'
driver.get(url)
time.sleep(1)

In [5]:
new_df_200 = pd.read_csv('new_df_200.csv')

In [6]:
new_df = pd.DataFrame()
restaurant_name_list = []
category_name_list = []
address_list = []
menu_list = []
price_list = []

In [7]:
## 인덱스 초기화 시켜줘야 돌아감

new_df_200.reset_index(inplace=True)
new_df_200.drop('index',axis=1,inplace=True)
new_df_200.head()

,Unnamed: 0,level_0,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소,검색여부
0,0,0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,한식,202203.971845,444853.761458,서울특별시,강남구,"강남대로114길18,",가능
1,1,1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트 강남역점,한식,202462.794327,443827.961945,서울특별시,강남구,"강남대로378,",가능
2,2,3,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,한식,203470.848439,447369.579852,서울특별시,강남구,"압구정로343,",가능
3,3,5,영업/정상,서울특별시 강남구 역삼동 769-1 삼환 아르누보,"서울특별시 강남구 역삼로 204, 삼환 아르누보 지하1층 B09,B10호 중 107...",알찬한끼,분식,203453.074369,443736.685792,서울특별시,강남구,"역삼로204,",가능
4,4,7,영업/정상,서울특별시 강남구 삼성동 155-11,"서울특별시 강남구 삼성로96길 11, 지상2층 (삼성동)",세계관,한식,204952.006872,445195.026116,서울특별시,강남구,"삼성로96길11,",가능


In [8]:
new_df_200.drop(['Unnamed: 0','level_0'],axis=1,inplace=True)

# 문제 1
개포로504, 37.5 시그니처 개포점 -> 전체 데이터 돌릴떈 안돌아가는데 하나만 돌릴떈 돌아감, 그렇다고해서 검색해서 바로 안넘어가는 케이스도 아님

-> 이거 말고 더 있을 수 있음
# 문제 2

만소당 -> test 데이터로 돌릴떈 프레임 넘어가고 정보도 잘 불러옴 근데 전체로 할떈 안불러옴 

-> 이거 말고 더 있을 수 있음 

# 문제3
세븐어클락이라는 식당 --> 테헤란로6길30, 세븐어클락(7O'CLOCK) 이라고 검색해서 검색 완료됌.

                      --> 근데 식당정보 크롤링할때 동일하게 검색했는데 안뜸

                      --> 근데 세븐어클락이라고 치면 결과 나옴, 주소도 같음. 

                      --> 테헤란로6길30, 세븐어클락 이라고 쳐도 나옴 ;; (7O'CLOCK) 포함해서 검색하면 안됌 

-> 이거는 그냥 노가다 떄려야할것같스빈ㄷ.
                      





In [11]:
new_df_200['사업장명'].unique()

array(['정식당', '채선당 샤브보트 강남역점', '수아당', '알찬한끼', '세계관', '분식을 품다', '식당 민홍',
       '제주그집', '만수정 강남점', '은서네 국내산 대패삼겹살', '냉동고집 삼성점', '리하베스트', '팡뮤제',
       '만소당', '꼬모 서울', '겐지(KENJI)', '논현136', '피양옥', "세븐어클락(7O'CLOCK)",
       '김재윤김치', '앰비언트 피크닉(AMBIENT PICNIK)', '쓰촨 마라', '스시 쥬고야', '조선육회',
       '브레드_히어_(Bread_Here_)', '후토루 역삼점', '블루플레임', '왕손피자 역삼점', '몽크스부처',
       '난바스낵 압구정로데오', '국신라면', '킹도리탕', '라이스박스 논현점', '포시애틀 은마상가점',
       '다이어메이커(Dyer Maker)', 'ETHOS 압구정', '가치있는 샐러드', '팀 홀튼 신논현역점',
       '옥된장 도곡점', '토종돼지 삼겹살전문', '압구정육식당', '마농 에 마농 마농와인', '일월분식', '그라도스',
       '일리있는', '갸또올리비에', '더블유(W)', '라장양갈비·양꼬치 무한리필', '경복', '한신 김치찌개',
       '치치 다이닝', '쇼미비치', '노슈가존', '도산포차', '도리토리 삼성직영점', '아맘돈까스 한티역점',
       '맘스피자 대치사거리점', '플로우라운지(FLOW LOUNGE)', '아이언케이스', '이목 스모크다이닝',
       '느린마을양조장 압구정로데오점', "진스토랑(Jin's torang)", '영희탕후루', '호시타코야끼 대치점',
       '에그앤플라워', '바르다김선생 압구정역점', '모티에(MOTIE)', '솔트 플러스 39(Salt Plus 39)',
       '청담 수', '타이거꼬마김밥(언주역점)', '로쿄 신논현', '도산정육블랙 청담본점', '신의주찹쌀순대',
       '이더웨이 쏘 굿

In [12]:
test_df = new_df_200.iloc[10:15,:]
test_df.reset_index(inplace=True)

In [14]:
# # 일단 검색결과 안나오고 바로 페이지가 넘어간다고 가정하고 

# for i in range(0,len(test_df)):
#     # 검색어를 네이버 url에 포함시켜 이동 + 검색 
    
#     # name = test_df['상세주소'][i] + ' ' + test_df['사업장명'][i]
#     name = '개포로504,' + ' ' + '37.5 시그니처 개포점'
#     driver.get('https://map.naver.com/p/search/{}'.format(name))

        
#     time.sleep(3)  

#     ## 화면이 안넘어가고 검색결과에 머물러 있을 때는 entryIframe 이 없음
#     ## 그래서 검색어 쳤을때 entryIframe 있으면 --> 화면이 넘어갔다는 뜻이고 
#     ## 없으면 아직 안넘어간 뜻이니깐 searchIframe 에서 클릭해서 넘어가야함 .
#     try :
#         if driver.find_elements(By.ID,'entryIframe') :
#             entryIframe = driver.find_element(By.ID,'entryIframe')
#             driver.switch_to.frame(entryIframe)
            
#             time.sleep(3) 
#             # 가게 이름, 변수로 지정
#             restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
#             category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
#             address = driver.find_element(By.CLASS_NAME,'LDgIH').text
#             if driver.find_elements(By.CLASS_NAME,'ihmWt') :
#                 menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
#                 price = driver.find_element(By.CLASS_NAME,'mkBm3').text
#             elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
#                 menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
#                 price = driver.find_element(By.CLASS_NAME,'Yrsei').text
#             else : 
#                 menu = '메뉴없음'
#                 price = "가격없음"
            
#         else : 
#             # searchIframe 프레임 

#                 searchIframe = driver.find_element(By.ID,'searchIframe')
#                 driver.switch_to.frame(searchIframe)


#                 ## 클릭 
#                 driver.find_element(By.CLASS_NAME,'YwYLL').click()
                
#                 # 기본프렝미 
#                 time.sleep(1)
#                 driver.switch_to.default_content()
            

#                 # entryIframe 프레임 
#                 entryIframe = driver.find_element(By.ID,'entryIframe')
#                 driver.switch_to.frame(entryIframe)

#                 time.sleep(3) 
#                 # 가게 이름, 변수로 지정
#                 restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
#                 category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
#                 address = driver.find_element(By.CLASS_NAME,'LDgIH').text
#                 if driver.find_elements(By.CLASS_NAME,'ihmWt') :
#                     menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
#                     price = driver.find_element(By.CLASS_NAME,'mkBm3').text
#                 elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
#                     menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
#                     price = driver.find_element(By.CLASS_NAME,'Yrsei').text
#                 else : 
#                     menu = '메뉴없음'
#                     price = "가격없음"
#     except Exception as e : 
#         restaurant_name = name 
#         category_name = '전처리필요'
#         address = '전처리필요'
#         menu = '메뉴없음'
#         price = "가격없음"
#         print(f"에러 메시지: {str(e)}")


#     ## 리스트 ㄱ 
#     restaurant_name_list.append(restaurant_name)
#     category_name_list.append(category_name)
#     address_list.append(address)
#     menu_list.append(menu)
#     price_list.append(price)
    
#     print('식당 이름:',restaurant_name)
#     print('식당 카테고리:',category_name)
#     print('주소:',address)
#     print('대표 메뉴:',menu)
#     print('가격:',price)
#     print('-----------------------------')
# naver_df_200 = pd.DataFrame({'식당이름':restaurant_name_list ,
#                              '업태구분' :category_name_list ,
#                              '주소' :address_list ,
#                              '메뉴': menu_list,
#                              '가격':price_list
#                              })

    

식당 이름: 37.5 시그니처 개포점
식당 카테고리: 브런치
주소: 서울 강남구 개포로 504 메트하임 17층
대표 메뉴: 정통 미국식 브런치
가격: 18,900원
-----------------------------
식당 이름: 37.5 시그니처 개포점
식당 카테고리: 브런치
주소: 서울 강남구 개포로 504 메트하임 17층
대표 메뉴: 정통 미국식 브런치
가격: 18,900원
-----------------------------
식당 이름: 37.5 시그니처 개포점
식당 카테고리: 브런치
주소: 서울 강남구 개포로 504 메트하임 17층
대표 메뉴: 정통 미국식 브런치
가격: 18,900원
-----------------------------
식당 이름: 37.5 시그니처 개포점
식당 카테고리: 브런치
주소: 서울 강남구 개포로 504 메트하임 17층
대표 메뉴: 정통 미국식 브런치
가격: 18,900원
-----------------------------
식당 이름: 37.5 시그니처 개포점
식당 카테고리: 브런치
주소: 서울 강남구 개포로 504 메트하임 17층
대표 메뉴: 정통 미국식 브런치
가격: 18,900원
-----------------------------


In [13]:
# 일단 검색결과 안나오고 바로 페이지가 넘어간다고 가정하고 

for i in range(0,len(test_df)):
    # 검색어를 네이버 url에 포함시켜 이동 + 검색 
    
    name = test_df['상세주소'][i] + ' ' + test_df['사업장명'][i]
    # name = '논현로159길10,' + ' ' + '만소당	'
    driver.get('https://map.naver.com/p/search/{}'.format(name))

        
    time.sleep(3)  

    ## 화면이 안넘어가고 검색결과에 머물러 있을 때는 entryIframe 이 없음
    ## 그래서 검색어 쳤을때 entryIframe 있으면 --> 화면이 넘어갔다는 뜻이고 
    ## 없으면 아직 안넘어간 뜻이니깐 searchIframe 에서 클릭해서 넘어가야함 .
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
            
            time.sleep(3) 
            # 가게 이름, 변수로 지정
            restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
            category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
            address = driver.find_element(By.CLASS_NAME,'LDgIH').text
            if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                price = driver.find_element(By.CLASS_NAME,'mkBm3').text
            elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                price = driver.find_element(By.CLASS_NAME,'Yrsei').text
            else : 
                menu = '메뉴없음'
                price = "가격없음"
            
        else : 
            # searchIframe 프레임 

                searchIframe = driver.find_element(By.ID,'searchIframe')
                driver.switch_to.frame(searchIframe)


                ## 클릭 
                driver.find_element(By.CLASS_NAME,'YwYLL').click()
                
                # 기본프렝미 
                time.sleep(1)
                driver.switch_to.default_content()
            

                # entryIframe 프레임 
                entryIframe = driver.find_element(By.ID,'entryIframe')
                driver.switch_to.frame(entryIframe)

                time.sleep(3) 
                # 가게 이름, 변수로 지정
                restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
                category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
                address = driver.find_element(By.CLASS_NAME,'LDgIH').text
                if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                    menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                    price = driver.find_element(By.CLASS_NAME,'mkBm3').text
                elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                    menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                    price = driver.find_element(By.CLASS_NAME,'Yrsei').text
                else : 
                    menu = '메뉴없음'
                    price = "가격없음"
    except Exception as e : 
        restaurant_name = name 
        category_name = '전처리필요'
        address = '전처리필요'
        menu = '메뉴없음'
        price = "가격없음"
        print(f"에러 메시지: {str(e)}")


    ## 리스트 ㄱ 
    restaurant_name_list.append(restaurant_name)
    category_name_list.append(category_name)
    address_list.append(address)
    menu_list.append(menu)
    price_list.append(price)
    
    print('식당 이름:',restaurant_name)
    print('식당 카테고리:',category_name)
    print('주소:',address)
    print('대표 메뉴:',menu)
    print('가격:',price)
    print('-----------------------------')
naver_df_200 = pd.DataFrame({'식당이름':restaurant_name_list ,
                             '업태구분' :category_name_list ,
                             '주소' :address_list ,
                             '메뉴': menu_list,
                             '가격':price_list
                             })

    

식당 이름: 냉동고집 삼성점
식당 카테고리: 육류,고기요리
주소: 서울 강남구 삼성로 517 1층 104호
대표 메뉴: 냉동삼겹살
가격: 15,000원
-----------------------------
식당 이름: 리하베스트
식당 카테고리: 카페,디저트
주소: 서울 강남구 역삼로 220 1층
대표 메뉴: 메뉴없음
가격: 가격없음
-----------------------------
식당 이름: 팡뮤제 베이커리
식당 카테고리: 베이커리
주소: 경기 김포시 풍무로 128 웰라움퍼펙트시티 1층
대표 메뉴: 아메리카노
가격: 4,800원
-----------------------------
식당 이름: 만소당
식당 카테고리: 돈가스
주소: 서울 강남구 논현로159길 10 104호
대표 메뉴: 돈까스
가격: 13,000원
-----------------------------
식당 이름: 꼬모 서울
식당 카테고리: 카페,디저트
주소: 서울 강남구 봉은사로74길 19 1층
대표 메뉴: 이탈리안 티라미수
가격: 6,500원
-----------------------------


['정식당', '채선당 샤브보트 강남역점', '수아당', '알찬한끼', '세계관', '분식을 품다', '식당 민홍',
       '제주그집', '만수정 강남점', '은서네 국내산 대패삼겹살', '냉동고집 삼성점', '리하베스트', '팡뮤제',
       '만소당', '꼬모 서울', '겐지(KENJI)', '논현136', '피양옥', "세븐어클락(7O'CLOCK)",

In [1]:
naver_df_200.head()

NameError: name 'naver_df_200' is not defined

In [ ]:
naver_df

,식당이름,업태구분,주소,메뉴,가격
0,정식당,한식,서울 강남구 강남대로114길 18 . 1층 정식당,소불고기주물럭,"16,000원"
1,정식당,한식,서울 강남구 강남대로114길 18 . 1층 정식당,소불고기주물럭,"16,000원"
2,채선당 샤브보트 강남역점,샤브샤브,서울 강남구 강남대로 378 . 지상1층,소고기 샤브샤브,"12,900원"
3,수아당 갤러리아백화점 명품관 WEST점,김밥,서울 강남구 압구정로 343,유부월남쌈,"18,000원"
4,알찬한끼,종합분식,서울 강남구 역삼로 204 삼환아르누보 지하1층 107호,알찬도시락,"5,000원"
5,세계관 삼성본점,한식,서울 강남구 삼성로96길 11 2층,투뿔 한우 곱도리탕,"32,000원"
6,분식을품다 한티점,종합분식,서울 강남구 역삼로 409 1층 102호,쌀 떡볶이,"5,900원"
7,식당민홍,일식당,서울 강남구 논현로146길 11 2층,오마카세 코스,"122,000원"
8,제주그집,한식,서울 강남구 논현로8길 11 지상1층,냉동삼겹살(150g),"14,000원"
9,만수정 강남점,"장어,먹장어요리",서울 강남구 강남대로 628 비율라빌딩 1층,민물장어 1kg ( 650~680g),"90,000원"
